In [1]:

# !pip3 install ffmpeg
# !pip3 install pydub
# !pip3 install matplotlib
# !pip3 install torch
# !pip3 install torchaudio
# !pip3 install librosa
# !pip3 install inflect
# !pip3 install deep_phonemizer



In [2]:
# !pip3 install deep_phonemizer


In [3]:
# import re
# import os 

# file= open("wavs.txt")
# file = file.readlines()

# for i, line in enumerate(file):
#     filename = re.findall(r'\d.wav.',line)
#     pattern = filename[0]
#     print(pattern)
    
#     replace = ''
#     if i <=8:
#         output_str = re.sub("\d.wav|", replace, line)
#     elif i <=98:
#         output_str = re.sub("\d\d.wav|", replace, line)
#     else:
#         output_str = re.sub("\d\d\d.wav|", replace, line)
#     output_str = output_str.replace("|","")
#     print(output_str)
#     path = os.path.join("/home/ali/Desktop/Namal/Idrak/trump/wavs_text",f'{i+1}.txt')
#     file = open(path,'w')
#     file = file.write(output_str)


In [4]:
# import inflect
# import re

            
            
           
# def cleaning_text_files (dir_path): 
    
#     paths = os.listdir(dir_path)
#     p = inflect.engine()
#     count = 0
#     for path in paths:
#         if count ==5:
#             break
#         path = os.path.join(dir_path,path)
#         with open(path,'r') as file:
#                     data = file.read()
#                     data = data.split()
#                     for index, word in enumerate(data):
#                         if word.isnumeric():
#                             word = int(word)
#                             changed_word = p.number_to_words(word)
#                             data[index]= changed_word
#                     new_data = ' '.join(data)
                    
#                     #Replacing Contractions with full words
#                     #specific
                    
#                     new_data = re.sub(r"won\'t", "will not", new_data)
#                     new_data = re.sub(r"can\'t", "can not", new_data)

#                     # general
#                     new_data = re.sub(r"n\'t", " not", new_data)
#                     new_data = re.sub(r"\'re", " are", new_data)
#                     new_data = re.sub(r"\'s", " is", new_data)
#                     new_data = re.sub(r"\'d", " would", new_data)
#                     new_data = re.sub(r"\'ll", " will", new_data)
#                     new_data = re.sub(r"\'t", " not", new_data)
#                     new_data = re.sub(r"\'ve", " have", new_data)
#                     new_data = re.sub(r"\'m", " am", new_data)
                    
#                     #Removing Puntuation Marks
#                     new_data = re.sub(r'[^\w\s]', '', new_data)
#                     file.close()
#         count+=1
#         with open(path,'w') as file:
#             file.write(new_data)
#             file.close()
    
# dir_path = "/home/ali/Desktop/Namal/Idrak/trump/wavs_text"
# cleaning_text_files(dir_path)
    
            
# !pip3 install torchaudio==0.12.0



In [5]:

import os
import re

import IPython.display as ipd
import librosa
import librosa.display

import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import matplotlib.pyplot as plt


In [6]:

# !pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html


In [7]:
torch.cuda.is_available()

True

In [8]:
torch.cuda.get_device_name()

'Tesla T4'

In [9]:
torch.cuda.memory_reserved()

0

In [10]:
torch.cuda.memory_allocated()

0

In [11]:
!pip3 install deep_phonemizer

  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00
  Created wheel for deep_phonemizer: filename=deep_phonemizer-0.0.19-py3-none-any.whl size=33299 sha256=c01aa085a0c51acdbb84dfa30d156f96973848279489eaa787c29a1999157860
  Stored in directory: /root/.cache/pip/wheels/c7/61/da/39b050eea5710fe32b1521b79de4a74d95896581230d30a37b
Successfully built deep_phonemizer
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
opentelemetry-api 1.17.0 requires importlib-metadata~=6.0.0, but you have importlib-metadata 5.2.0 which is incompatible.
pym

In [12]:

import os
import re




dir_path = "/kaggle/input/trump-voice-deepfake/full_wavs1"
paths = os.listdir(dir_path)
paths = sorted(paths, key=lambda s: int(re.search(r'\d+', s).group()))


def audio_preprocessing(audio_file_path):
    
    metadata = torchaudio.info(audio_file_path)
    waveform,sample_rate = torchaudio.load(audio_file_path)
    # waveform,sample_rate = librosa.load(audio_file_path,sr=None)
    waveform = waveform.mean(dim=0)
   
    waveform = torch.tensor(waveform)
    n_fft = 1024
    win_length = None
    hop_length = 512
    n_mels = 80

    mel_spectrogram = T.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        win_length=win_length,
        hop_length=hop_length,
        center=True,
        pad_mode="reflect",
        power=2.0,
        norm="slaney",
        onesided=True,
        n_mels=n_mels,
        mel_scale="htk",
    )
    melspec = mel_spectrogram(waveform)

    return waveform,sample_rate,melspec


melspecs = []
melspec_lengths =[]
melspec_times = []

for idx,path in enumerate(paths):
    audio_file_path = os.path.join(dir_path,path)
    waveform,sample_rate,melspec = audio_preprocessing(audio_file_path)
    melspecs.append(melspec)
    melspec_lengths.append(torch.tensor(melspec.shape[1]))
    

    



/tmp/ipykernel_23/2454150262.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  waveform = torch.tensor(waveform)
/opt/conda/lib/python3.10/site-packages/torchaudio/transforms/_transforms.py:611: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


In [13]:

dir_path = "/kaggle/input/trump-voice-deepfake/full_wavs_text"
paths = os.listdir(dir_path)
paths = sorted(paths, key=lambda s: int(re.search(r'\d+', s).group()))
bundle = torchaudio.pipelines.TACOTRON2_WAVERNN_PHONE_LJSPEECH

processor = bundle.get_text_processor()

text_tensors = []
text_lengths  =[]
with torch.inference_mode():
    for path in paths:
        text_file_path = os.path.join(dir_path,path)
        file = open(text_file_path,'r')
        text = file.read()
        processed, lengths = processor(text)
        text_tensors.append(processed)
        text_lengths.append(lengths)

100%|██████████| 63.6M/63.6M [00:03<00:00, 21.3MB/s]


In [14]:

melspec_lengths = torch.stack(melspec_lengths)


In [15]:
melspec_lengths = melspec_lengths.squeeze()


In [16]:
text_lengths  =torch.stack(text_lengths)

In [17]:
text_lengths = text_lengths.squeeze()

In [18]:
# text_indices = torch.argsort(text_lengths,dim=0)/

In [19]:
for idx,tensor in enumerate(melspecs):
    padding_req = melspec_lengths.max()-tensor.shape[1]
    padded_tensor = torch.nn.functional.pad(tensor,(0,padding_req,0,0),mode='constant',value=0)
    melspecs[idx] = padded_tensor

melspecs = torch.stack(melspecs)
melspecs.shape

torch.Size([598, 80, 5676])

In [20]:
melspecs = melspecs.squeeze()

In [21]:


melspecs.shape

torch.Size([598, 80, 5676])

In [22]:
symbols = "_-!'(),.:;? abcdefghijklmnopqrstuvwxyz"

look_up = {s: i for i, s in enumerate(symbols)}
symbols = set(symbols)


def text_to_sequence(text):
    text = text.lower()
    return [look_up[s] for s in text if s in symbols]


In [23]:
# !pip3 install deep_phonemizer

In [24]:

# padding the Tensors to the max_length tensor
for idx,tensor in enumerate(text_tensors):
    padding_req = max(text_lengths)-tensor.shape[1]
    padded_tensor = torch.nn.functional.pad(tensor,(0,padding_req,0,0),mode='constant',value=0)
    text_tensors[idx] = padded_tensor


text_tensors = torch.stack(text_tensors)

In [25]:

text_tensors = torch.tensor(text_tensors,dtype=torch.int32)


/tmp/ipykernel_23/1352164243.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_tensors = torch.tensor(text_tensors,dtype=torch.int32)


In [26]:
text_tensors = text_tensors.squeeze()


In [27]:
text_lengths,sort_indices = text_lengths.sort(0,descending=True)
text_tensors= text_tensors[sort_indices]


In [28]:
import torch.nn.utils.rnn as rnn_utils



packed_text = rnn_utils.pack_padded_sequence(text_tensors, text_lengths, batch_first=True)
# packed_mel_specs = rnn_utils.pack_padded_sequence(mel_specs, melspec-lengths, batch_first=True)

In [29]:
from torch.utils.data import DataLoader



In [30]:
from sklearn.model_selection import train_test_split
mel_train,mel_test,mel_length_train,mel_length_test,text_train,text_test,text_length_train,text_length_test  =train_test_split(melspecs,
                                                                        melspec_lengths,
                                                                       text_tensors,text_lengths,
                                                            test_size=0.3, random_state=42            )


In [31]:
batch =5
learning_rate = 0.001
epochs =5

In [32]:
mel_train_data = DataLoader(mel_train,batch_size=batch) # tenosrs of Shape [80,28680]

mel_length_train_data = DataLoader(mel_length_train,batch_size=batch) # Tensors of shape [,2]

text_train_data  =DataLoader(text_train, batch)
text_length_train_data = DataLoader(text_length_train,batch_size=batch)

In [33]:
mel_test_data = DataLoader(mel_test,batch_size=batch) # tenosrs of Shape [80,28680]

mel_length_test_data = DataLoader(mel_length_test,batch_size=batch) # Tensors of shape [,2]

text_test_data  =DataLoader(text_test, batch_size=batch)
text_length_test_data = DataLoader(text_length_test,batch_size=batch)

Setting Up The GPU Setting -- CUDA

In [34]:
# from torch.nn.parallel import DataParallel

In [35]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
from torchaudio.models import Tacotron2
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# bundle = torchaudio.pipelines.TACOTRON2_WAVERNN_PHONE_LJSPEECH
# tacotron2 = bundle./
# Tacotron2.forward(tacotron2,text_tensors,text_lengths,melspecs,melspec_lengths)


# model = Tacotron2()

# model  = model.to(device)
# if torch.cuda.device_count() > 1:
#     model = DataParallel(model,device_ids=[0,1],dim=0)



In [37]:
torch.cuda.empty_cache()

In [38]:
print(melspecs.shape,melspec_lengths.shape,text_tensors.shape,text_lengths.shape)

torch.Size([598, 80, 5676]) torch.Size([598]) torch.Size([598, 903]) torch.Size([598])


In [39]:



# Create the optimizer
# mae_loss = torch.nn.MSELoss()
# stop_tokens_loss = torch.nn.BCEWithLogitsLoss()


# optimizer = torch.optim.Adam(params=model.parameters(), # parameters of target model to optimize
#                             lr=learning_rate)

In [40]:
torch.cuda.memory_reserved()

0

In [41]:
# !pip3 install tqdm

In [42]:
# import gc
# import torch.nn.utils.rnn as rnn_utils



# train_loss_values = []
# test_loss_values = []
# epoch_count = []

# from tqdm import tqdm
# # torch.cuda.empty_cache()
# # gc.collect()


# accumulation_steps  = 5



# for epoch in tqdm(range(epochs)):
#     count = 0
#     model.train()
#     for mel_specs,mel_lens,text,text_lens in zip(mel_train_data,
#                                                  mel_length_train_data,
#                                                  text_train_data,
#                                                  text_length_train_data):
        
        
        

#         count+=1
        
#         text_lens,sort_indices = text_lens.sort(0,descending=True)
#         text = text[sort_indices]
# #         mel_lens, sort_indices = torch.sort(mel_lens, descending=True)
#         mel_specs = mel_specs[sort_indices]
#         mel_lens = mel_lens[sort_indices]
# #         print(mel_lens)

# #         text_lens = text_lens[sort_indices]
# #         print(text_lens.shape)
# #         text = text[sort_indices]
#         # print(mel_specs.shape)
#         # print(mel_train_data)
#         stop_lst = [melspec[1] for melspec in mel_specs]
        
#         stop_targets = torch.stack(stop_lst)
#         # print(stop_targets.shape)
       
#         stop_targets  = stop_targets.to(device)
        

#         text = text.to(device)
#         # print(torch.cuda.memory_allocated())
        
#         text_lens = text_lens.to(device)
#         # print(torch.cuda.memory_allocated())

#         mel_specs = mel_specs.to(device)
#         # print(torch.cuda.memory_allocated())

#         mel_lens = mel_specs.to(device)
#         # print(torch.cuda.memory_allocated())
# #         packed_text = rnn_utils.pack_padded_sequence(text, text_lens, batch_first=True)
# #         packed_mel_specs = rnn_utils.pack_padded_sequence(mel_specs, mel_lens, batch_first=True)
# #         print(packed_text)
        
#         try:
#             pre_specs, post_specs, stop_tokens_pred,_ = model(text,text_lens,mel_specs,mel_lens)
            
#         except RuntimeError as e:
            
#             if "CUDA out of memory" in str(e):
#                 # handle out of memory error
                
                
                
#                 pre_specs, post_specs, stop_tokens_pred,_ = model(text,text_lens,mel_specs,mel_lens)
#             else:
#                 raise e

      
        
        
#         mel_loss= mae_loss(pre_specs,post_specs) 

#         stop_loss = stop_tokens_loss(stop_tokens_pred,stop_targets)
        
        
#         total_loss = mel_loss +stop_loss
#         total_loss  = total_loss / accumulation_steps
#         total_loss.backward()
#         if count % accumulation_steps == 0:
#           # print("hellloasd")
#             optimizer.zero_grad()
#             optimizer.step()
#             count = 0
#           # print("gradient Acculated")
#     model.eval()
#     with torch.inference_mode():
#         for mel_specs,mel_lens,text,text_lens in zip(mel_test_data,
#                                                     mel_length_test_data,
#                                                     text_test_data,
#                                                     text_length_test_data):
            
#             mel_lens, sort_indices = torch.sort(mel_lens)
#             mel_specs = mel_specs[sort_indices]
# #             print(_lens)
            
#             text_lens = text_lens[sort_indices]
#             text = text[sort_indices]
# #             print(text_lens.shape)
# #             packed_text = rnn_utils.pack_padded_sequence(text, text_lens, batch_first=True)
# #             packed_mel_specs = rnn_utils.pack_padded_sequence(mel_specs, mel_lens, batch_first=True)
# #             print(packed_text)
# #             packed_text.data, packed_text.batch_sizes, packed_mel_specs.data, packed_mel_specs.batch_sizes
#             stop_lst = [melspec[1] for melspec in mel_specs]

           
        
#             stop_targets_test = torch.stack(stop_lst)

#             stop_targets_test  = stop_targets_test.to(device)
            
            

#             text = text.to(device)
#             # print(torch.cuda.memory_allocated())
            
#             text_lens = text_lens.to(device)
#             # print(torch.cuda.memory_allocated())

#             mel_specs = mel_specs.to(device)
#             # print(torch.cuda.memory_allocated())

#             mel_lens = mel_specs.to(device)
#             # print(torch.cuda.memory_allocated())

#             pre_specs, post_specs, stop_tokens_pred,_ = model(text,text_lens,mel_specs,mel_lens)
#             mel_loss_test= mae_loss(pre_specs,post_specs) 

#             stop_loss_test = stop_tokens_loss(stop_tokens_pred,stop_targets_test)
        
        
#             test_loss = mel_loss_test +stop_loss_test
#         # put the model in evaluation mode for testing (inference)
   
        

    
#     # torch.cuda.empty_cache()
   
#     # # if epoch % 10 == 0:
#     epoch_count.append(epoch)
#     total_loss = total_loss.cpu()
#     train_loss_values.append(total_loss.detach().cpu().numpy())
#     test_loss_values.append(test_loss.detach().cpu().numpy())
#     print(f"Epoch: {epoch} |  Train Loss: {total_loss} | Test Loss {test_loss} ")
    
        
       

In [43]:
num_of_gpus = torch.cuda.device_count()
print(num_of_gpus)

2


In [44]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |